<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab4/lab5_NMT_with_Attention_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2
!git pull
%cd lab4

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ai-intensive2
Already up to date.
/content/drive/MyDrive/ai-intensive2/lab4


# Lab 5 : Neural Machine Translation with Attention Mechanism

@copyright:
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.

---


## 목차

1. Neural Machine Translation
    - Task
    - Data
    - Model
2. Attention Mechanism
    - Attention
    - Implementation
    - Training
    - Evaluation

## 1. Neural Machine Translation (Data & Model)


### Task

- 이번 챕터에서는 Neural Machine Translation (NMT)에 대해 다루며, Seq2Seq 모델에 대해 알아본다.
- 특히, 지난시간 학습한 Seq2Seq 모델에 Attention Mechanism을 추가하여 NMT를 구현해본다.

- 본 실습에서는, French to English 번역을 수행하는 NMT 모델을 구현한다.

[KEY: > input, = target, < output]

\ > il est en train de peindre un tableau .

\ = he is painting a picture .

\ < he is painting a picture .

- 지난 시간에 학습한 Seq2Seq 모델은 Encoder와 Decoder로 구성되어 있으며, Encoder는 입력 문장을 의미 벡터로 변환하고, Decoder는 의미 벡터를 입력으로 받아 번역 문장을 생성한다.
- 이 모델을 발전시키기 위해, Attention Mechanism을 추가한다.

### Data

https://www.manythings.org/anki/

- 본 실습에서는, 위의 링크에서 다운로드 받은 Korean to English 번역 데이터를 사용한다.


In [11]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

- 언어의 각 단어를 one-hot vector로 표현한다.

- 나중에 network의 입력과 출력으로 사용하기 위해, 각 단어를 index로 표현한다.
- word2idx, idx2word dictionary를 가지는 helper class `Lang`을 정의한다.

In [12]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [13]:
# File consist of Unicode, so make it to String
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    return s.strip()


- Datafile을 읽고, line별로 나눈다. 또한 line을 pari 별로 나눈다.
- Data는 English->French로 되어있으므로, reverse flag를 통해 French->English로 바꿀 수 있다.

In [14]:
def readLangs(lang1, lang2, reverse):
    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [15]:
#Filter out too short sentence
#Also process apostrophes

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [16]:
def prepareData(lang1, lang2, reverse):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    pairs = filterPairs(pairs)

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'kor', True)
print(pairs[42])

Counted words:
kor 149
eng 131
['그는 흰 옷을 입고 있다 .', 'she is dressed in white .']


- torch module을 활용하여, Prepare한 data를 Tensor로 변환하고, DataLoader를 정의한다.

In [17]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'kor', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

### Model

- 지난 시간에 학습한 Seq2Seq 모델은 Encoder와 Decoder로 구성되어 있으며, Encoder는 입력 문장을 의미 벡터로 변환하고, Decoder는 의미 벡터를 입력으로 받아 번역 문장을 생성한다.

- 본 실습에서는, torch nn module을 사용하여 Encoder와 Decoder를 구현한다.

### Encoder
![Encoder RNN](https://github.com/dh610/ai-intensive2/blob/main/lab4/data/encoder_network.png?raw=1)

In [27]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # Fill this cell to implement encoder RNN
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        # Fill this cell to implement forward pass of encoder RNN
        output = self.dropout(self.embedding(input))
        output, hidden = self.gru(output)

        return output, hidden

### Decoder
![Decoder RNN](https://github.com/dh610/ai-intensive2/blob/main/lab4/data/decoder_network.png?raw=1)

In [19]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        # Fill this cell to implement decoder RNN
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden) #Fill here
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)

        return decoder_outputs, decoder_hidden, None

    def forward_step(self, input, hidden):
        #Fill this cell to implement forward step of decoder RNN
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

## 2. Attention Mechanism

### Attention

- En-Decoder 사이 latent vector만 전달되는 경우, 해당 vector에 모든 정보가 담겨있어야 한다.

- Attention Mechanism은 Decoder가 출력을 생성할 때, Encoder의 모든 입력을 고려하는 것이 아니라, 해당 출력에 대해 필요한 입력만을 "집중"(Attention)하도록 한다.

![Attention Mechanism](https://github.com/dh610/ai-intensive2/blob/main/lab4/data/attention_mechanism.png?raw=1)

- 먼저, Attention weight를 계산한다.

- 여기에 Encoder의 출력을 곱하여, Attention value를 계산한다.

- Attention value는 입력 sequence의 특정 부분에 집중하여, 해당 부분의 정보를 Decoder에 전달한다.

### Additive Attention

- [Additive Attention](https://arxiv.org/pdf/1409.0473.pdf)(Bahdanau Attention)은 다음과 같은 식으로 계산된다.

$$ score(s_{t}, h_{s}) = v_{a}^{T} tanh(W_{a}s_{t} + U_{a}h_{s}) $$


In [20]:
class AdittiveAttention(nn.Module):
    def __init__(self, hidden_size):
        super(AdittiveAttention, self).__init__()
        self.hidden_size = hidden_size
        #""" Fill this cell to implement additive attention, the Wa, Ua, Va
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, key):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(key))) #fill this cell, calculate scores
        scores = scores.squeeze(2).unsqueeze(1) # (batch_size, 1, seq_len)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, key)

        return context, weights

### Implementation

해당 Attention module을 적용하여, Decoder를 다시 작성한다.

![Decoder RNN with Attention](https://github.com/dh610/ai-intensive2/blob/main/lab4/data/attention-decoder-network.png?raw=1)


In [21]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = AdittiveAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        """ Fill this cell to implement forward step of decoder RNN with attention
            See above cells for reference"""

        embedded = self.dropout(self.embedding(input))
        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat([embedded, context], dim = 2)
        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

### Training

- 구현한 Encoder, Decoder 모델을 학습시킨다.
- Dataloader를 사용한다.

In [22]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        #Zero the gradients
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        #Encoder
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        #Decoder
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        #Loss
        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        # Update parameters with optimizers
        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [23]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100):
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('Epoch %d %.4f' % (epoch, print_loss_avg))

In [24]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [25]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [28]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

evaluateRandomly(encoder, decoder)
# train(train_dataloader, encoder, decoder, 50, print_every=5)

Counted words:
kor 149
eng 131
> 그녀는 아름답다 .
= she is beautiful .
< playing dark-skinned am playing real really real real same too

> 그분은 절대 그런 걸 할 사람이 아니에요 .
= he is above doing such a thing .
< playing very next really he real real same am married

> 그는 서핑에 정말 미쳐있다 .
= he is really crazy about surfing .
< playing he afternoon i real same month person go real

> 그는 학급에서 가장 둔한 아이이다 .
= he is the dumbest kid in the class .
< very i real bank real real same month same we

> 그는 출생지로 따지면 독일인입니다 .
= he is german by birth .
< playing very next next employed too too a a easygoing

> 그 사람은 미혼이에요 .
= he is unmarried .
< job job tomorrow am friends longer responsibility dumbest dumbest married

> 그는 출생지로 따지면 독일인입니다 .
= he is german by birth .
< playing very next really same friends alone real same am

> 나는 인간이야 .
= i am human .
< playing i easygoing dressed beaut